In [9]:
import pickle
import pdb
import codecs
import re
import sys
import math

import numpy as np

import tensorflow as tf
from Batch import BatchGenerator
from bilstm_crf import Model
from utils import *

In [10]:
with open('../Bosondata.pkl', 'rb') as inp:
	word2id = pickle.load(inp)
	id2word = pickle.load(inp)
	tag2id = pickle.load(inp)
	id2tag = pickle.load(inp)
	x_train = pickle.load(inp)
	y_train = pickle.load(inp)
	x_test = pickle.load(inp)
	y_test = pickle.load(inp)
	x_valid = pickle.load(inp)
	y_valid = pickle.load(inp)
print("train len:",len(x_train))
print("valid len:",len(x_valid))
print("test len:",len(x_test))
print("word2id len", len(word2id))
print('Creating the data generator ...')

train len: 10721
valid len: 2681
test len: 3351
word2id len 3435
Creating the data generator ...


In [11]:
data_train = BatchGenerator(x_train, y_train, shuffle=True)
data_valid = BatchGenerator(x_valid, y_valid, shuffle=False)
data_test = BatchGenerator(x_test, y_test, shuffle=False)
print('Finished creating the data generator.')

Finished creating the data generator.


In [12]:
data_train.x.shape

(10721, 60)

In [13]:
epochs = 31
batch_size = 32

config = {}
config["lr"] = 0.001
config["embedding_dim"] = 100 # 词向量的维度
config["sen_len"] = len(x_train[0]) # 句子长度
config["batch_size"] = batch_size # 批次大小
config["embedding_size"] = len(word2id)+1 # 有多少个词向量
config["tag_size"] = len(tag2id) # 有多少个标签
config["pretrained"]=False # 预训练

In [14]:
embedding_pre = []

print("use pretrained embedding")
config["pretrained"]=True
word2vec = {}
with codecs.open('vec.txt','r','utf-8') as input_data:   
    for line in input_data.readlines():
        word2vec[line.split()[0]] = list(map(eval,line.split()[1:]))

unknow_pre = []
unknow_pre.extend([1]*100)
embedding_pre.append(unknow_pre) #wordvec id 0
for word in id2word:
    if word in word2vec:
        embedding_pre.append(word2vec[word])
    else:
        embedding_pre.append(unknow_pre)

embedding_pre = np.asarray(embedding_pre)

use pretrained embedding


In [15]:
embedding_pre

array([[ 1.      ,  1.      ,  1.      , ...,  1.      ,  1.      ,
         1.      ],
       [ 0.087197, -0.083435,  0.057956, ...,  0.045114, -0.08465 ,
         0.111534],
       [ 0.084424, -0.118023,  0.135428, ...,  0.273897,  0.150512,
        -0.172031],
       ...,
       [ 0.163283,  0.341556,  0.380325, ..., -0.099092,  0.167281,
        -0.46092 ],
       [-0.233846, -0.435442,  0.366504, ..., -0.282544, -0.129922,
         0.357925],
       [-0.578525, -0.012686, -0.060288, ...,  0.013066, -0.469461,
        -0.127309]])